In [88]:
import math

from keras.layers import *
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
 
columns = ['Speed','m0c','m1c','acc','lat','lon']

boat_curved = pd.read_csv("Data/Anomalous_boat_data.csv")
boat_curved= boat_curved.drop(columns=["Unnamed: 0", "heading"])    
boat_curved['degrees'][0] = 250
boat_curved= boat_curved[:6600]
scaler = StandardScaler()
anomalous_data = scaler.fit_transform(boat_curved.drop(columns=['degrees']))
anomalous_df = pd.DataFrame(anomalous_data, columns=columns)

def add_noise(st_devs):
    data = np.ones(shape=(6600,6))
    for i in range(6600):
        for j, value in enumerate(columns):
            v = anomalous_data[i][j]
            noise = np.random.normal(v, st_devs[j])
            data[i][j] = v + noise
    return data

columns_new = ['Speed', 'm0c', 'm1c', 'acc', 'lat', 'lon', 'sin', 'cos']

devs = [0.5, 1, 1, 2, 0.03, 0.03]
noisy_data = add_noise(devs)
zeros = np.zeros(shape=(6600,8))
zeros[:noisy_data.shape[0], :noisy_data.shape[1]] = noisy_data
noisy_data = zeros
noisy_df = pd.DataFrame(noisy_data, columns=columns_new)



id_list = []
def sin_cos():
    sins = []
    cosins = []
    for i, degree in enumerate(boat_curved["degrees"]):
        value = degree + np.random.normal(0, 2)
        sin = np.sin(np.deg2rad(value))
        cos = np.cos(np.deg2rad(value))
        id_list.append(sin**2+cos**2)
        sins.append(sin)
        cosins.append(cos)
    return sins, cosins    

sin,cos = sin_cos()
noisy_df['sin'] = sin
noisy_df['cos'] = cos

lenght = 10
for i in range(lenght):
    new_data = pd.DataFrame(add_noise(devs), columns=columns)
    sin, cos = sin_cos()
    new_data['sin'] = sin
    new_data['cos'] = cos
    noisy_df = noisy_df.append(new_data) 
# 
# plt.plot(noisy_df['sin'][6620:13240])
# plt.plot(noisy_df['cos'][6620:13240])
# plt.title("Sins-Cosins")
# plt.show()
# 
# plt.plot(id_list)
# plt.title("IDLIST")
# plt.show()
# 
# plt.plot(boat_curved['degrees'][:6620])
# plt.title("DEGREES")
# plt.show()


print(noisy_df)



         Speed       m0c       m1c       acc       lat       lon       sin  \
0     5.621329  0.289131 -0.312179 -2.135119 -3.548943  0.897262 -0.928684   
1     5.138855 -0.821228  1.500425  2.775360 -3.483321  0.875290 -0.915923   
2     6.231602 -1.660474  2.293065 -3.078832 -3.475313  0.763078 -0.924624   
3     4.697377  1.195294 -1.928085 -1.876235 -3.434160  0.765237 -0.919457   
4     4.905521  2.426659 -1.390575  3.904188 -3.454338  0.816576 -0.924536   
...        ...       ...       ...       ...       ...       ...       ...   
6595 -1.529504 -0.027810 -0.889648 -0.438932 -2.976241  4.414905  0.987395   
6596 -0.472938 -0.501245  1.534650  0.101797 -2.975422  4.426089  0.977915   
6597  0.276292  1.965984 -2.097410 -2.520142 -3.022935  4.434338  0.987892   
6598 -1.561877  1.110594 -0.095172 -4.424351 -2.954052  4.495397  0.999431   
6599 -1.726300 -1.457063 -1.140267 -0.872076 -3.001786  4.454672  0.985104   

           cos  
0    -0.370873  
1    -0.401355  
2    -0.3808

In [89]:
noisy_df.to_csv("Data/Boat_anomalous_Big.csv")